In [161]:
import pandas as pd

# Задача 1

При подключении новых проектов обычно требуется рассчитать метрики, чтобы корректно настроить
систему фрод-мониторинга исходя из этих данных. Необходимо составить SQL-запрос, чтобы получить таблицу с
полями:

● project - ID проекта и его название в одном поле через точку и пробел. Например: “15029. OVERWATCH”.

● successful_transactions - Число успешных платежей.

● successful_transactions_amount - Сумма успешных транзакций в у.е.

● average_check - Средний чек в проекте в у.е. (не медиана, а просто AVG от всех успешных платежей)

● max_amount_for_1_user - Максимальная сумма платежей для одного пользователя в заданном периоде в
у.е.

● TOP3_most_popular_ps - 3 самые популярные платежные системы в проекте по числу успешных платежей.

● TOP3_banks - 3 самых популярных банка по числу успешных платежей в проекте

In [162]:
Card_payments = pd.read_excel('Card_payments_2.xlsx')

In [163]:
Payments = pd.read_excel('Payments_3.xlsx')

In [164]:
Games = pd.read_excel('Games.xlsx')

In [165]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
Payments.to_sql('Payments', con=engine, index=False)
Games.to_sql('Games', con=engine, index=False)
Card_payments.to_sql('Card_payments', con=engine, index=False)

25

РЕШЕНИЕ 1

In [174]:
sql_code = """
            

                    SELECT DISTINCT project, 
                           successful_transactions,
                           successful_transactions_amount,
                           average_check,
                           sum_amount_nick,
                           TOP3_most_popular_ps,
                           TOP3_banks
                    FROM
                    (
                        SELECT *, 
                               project_id || '. ' || game_name AS project
                        FROM Payments
                        JOIN Games
                        ON Games.game_id = Payments.project_id
                    ) AS Payments_Games
                    LEFT JOIN
                    (
                        SELECT project_id, 
                               COUNT(*) AS successful_transactions,
                               SUM(amount) AS successful_transactions_amount,
                               AVG(amount) AS average_check
                        FROM Payments
                        WHERE status = '3' AND test_transaction = '0'
                        GROUP BY project_id
                        UNION ALL
                        SELECT DISTINCT project_id, 
                               0 AS successful_transactions,
                               0 AS successful_transactions_amount,
                               0 AS average_check
                        FROM Payments
                        WHERE project_id NOT IN
                        (
                            SELECT DISTINCT project_id
                            FROM Payments
                            WHERE status = '3' AND test_transaction = '0'
                        )
                    ) AS table_successful_transactions
                    ON table_successful_transactions.project_id = Payments_Games.project_id
                    LEFT JOIN
                    (
                        SELECT project_id, MAX(sum_amount_nick) AS sum_amount_nick
                        FROM
                        (
                            SELECT project_id, nick, SUM(amount) AS sum_amount_nick
                            FROM Payments
                            WHERE status = '3' AND test_transaction = '0'
                            GROUP BY project_id, nick
                        )
                        GROUP BY project_id
                        UNION ALL
                        SELECT DISTINCT project_id, 0 AS sum_amount_nick
                        FROM Payments
                        WHERE project_id NOT IN
                        (
                            SELECT DISTINCT project_id
                            FROM Payments
                            WHERE status = '3' AND test_transaction = '0'
                        )
                    ) AS table_sum_amount_nick
                    ON table_sum_amount_nick.project_id = Payments_Games.project_id
                    LEFT JOIN
                    (
                        SELECT project_id, 
                               GROUP_CONCAT(payment_system, ', ') AS TOP3_most_popular_ps
                        FROM 
                        (
                            SELECT project_id, payment_system
                            FROM
                            (
                                SELECT project_id, 
                                       payment_system, 
                                       COUNT(*) AS num_pays, 
                                       ROW_NUMBER() OVER(PARTITION BY project_id) AS rownum
                                FROM Payments
                                WHERE status = '3' AND test_transaction = '0'
                                GROUP BY project_id, payment_system
                                ORDER BY num_pays DESC
                            )
                            WHERE rownum < 4
                        )
                        GROUP BY project_id
                        UNION ALL
                        SELECT DISTINCT project_id, 
                                        0 AS TOP3_most_popular_ps
                        FROM Payments
                        WHERE project_id NOT IN
                        (
                            SELECT DISTINCT project_id
                            FROM Payments
                            WHERE status = '3' AND test_transaction = '0'
                        )
                    ) table_TOP3_most_popular_ps
                    ON table_TOP3_most_popular_ps.project_id = Payments_Games.project_id
                    LEFT JOIN
                    (
                        SELECT project_id, 
                        GROUP_CONCAT(card_bank, ', ') AS TOP3_banks
                        FROM 
                        (
                            SELECT project_id, card_bank
                            FROM
                            (
                                SELECT project_id, 
                                       card_bank, 
                                       COUNT(*) AS num_pays, 
                                       ROW_NUMBER() OVER(PARTITION BY project_id) AS rownum
                                FROM Payments
                                LEFT JOIN Card_payments
                                ON Card_payments.payment_id = Payments.id
                                WHERE status = '3' AND test_transaction = '0'
                                GROUP BY project_id, card_bank
                                ORDER BY num_pays DESC
                            )
                            WHERE rownum < 4
                        )
                        GROUP BY project_id
                        UNION ALL
                        SELECT DISTINCT project_id, 
                                        0 AS TOP3_banks
                        FROM Payments
                        WHERE project_id NOT IN
                        (
                            SELECT DISTINCT project_id
                            FROM Payments
                            WHERE status = '3' AND test_transaction = '0'
                        )
                    ) table_TOP3_banks
                    ON table_TOP3_banks.project_id = Payments_Games.project_id
                    

        """
temp = pd.read_sql(sql_code, engine)
temp

,project,successful_transactions,successful_transactions_amount,average_check,sum_amount_nick,TOP3_most_popular_ps,TOP3_banks
0,156. wc,8,553,69.125000,311,"t, x, m","sber, alfa, vtb"
1,157. wot,0,0,0.000000,0,0,0
2,161. worms,0,0,0.000000,0,0,0
3,164. sims,9,994,110.444444,500,"x, m, t","sber, vtb, alfa"
4,165. dota,0,0,0.000000,0,0,0


РЕШЕНИЕ 2

In [180]:
sql_code = """
            
            
            
            
                    WITH t_successful AS
                    (
                        SELECT project,
                               SUM(successful_transactions) AS successful_transactions,
                               SUM(successful_transactions_amount) AS successful_transactions_amount,
                               CAST(SUM(successful_transactions_amount) AS FLOAT)/SUM(successful_transactions) AS average_check,
                               MAX(sum_amount_nick) AS sum_amount_nick,
                               project_id
                        FROM
                        (
                            SELECT project_id || '. ' || game_name AS project, 
                                   COUNT(*) AS successful_transactions,
                                   SUM(amount) AS successful_transactions_amount,
                                   AVG(amount) AS average_check,
                                   SUM(amount) AS sum_amount_nick,
                                   project_id
                            FROM Payments
                            JOIN Games
                            ON Games.game_id = Payments.project_id
                            WHERE status = '3' AND test_transaction = '0'
                            GROUP BY project_id, nick
                        )
                        GROUP BY project
                    )
                    

                    
                    SELECT project, 
                           successful_transactions,
                           successful_transactions_amount, 
                           average_check, 
                           sum_amount_nick,
                           TOP3_most_popular_ps,
                           TOP3_banks
                    FROM t_successful
                    
                    LEFT JOIN
                    (
                        SELECT project_id, 
                        GROUP_CONCAT(card_bank, ', ') AS TOP3_banks
                        FROM 
                        (
                            SELECT project_id, card_bank
                            FROM
                            (
                                SELECT project_id, 
                                       card_bank, 
                                       COUNT(*) AS num_pays, 
                                       ROW_NUMBER() OVER(PARTITION BY project_id) AS rownum
                                FROM Payments
                                LEFT JOIN Card_payments
                                ON Card_payments.payment_id = Payments.id
                                WHERE status = '3' AND test_transaction = '0'
                                GROUP BY project_id, card_bank
                                ORDER BY num_pays DESC
                            )
                            WHERE rownum < 4
                        )
                        GROUP BY project_id
                    ) AS table_TOP3_banks
                    ON table_TOP3_banks.project_id = t_successful.project_id
                    
                    LEFT JOIN
                    (
                        SELECT project_id, 
                               GROUP_CONCAT(payment_system, ', ') AS TOP3_most_popular_ps
                        FROM 
                        (
                            SELECT project_id, payment_system
                            FROM
                            (
                                SELECT project_id, 
                                       payment_system, 
                                       COUNT(*) AS num_pays, 
                                       ROW_NUMBER() OVER(PARTITION BY project_id) AS rownum
                                FROM Payments
                                WHERE status = '3' AND test_transaction = '0'
                                GROUP BY project_id, payment_system
                                ORDER BY num_pays DESC
                            )
                            WHERE rownum < 4
                        )
                        GROUP BY project_id
                    ) AS table_TOP3_most_popular_ps
                    ON table_TOP3_most_popular_ps.project_id = t_successful.project_id
                    
                    UNION ALL
                    SELECT DISTINCT project_id || '. ' || game_name AS project, 
                           0 AS successful_transactions,
                           0 AS successful_transactions_amount,
                           0 AS average_check,
                           0 AS sum_amount_nick,
                           0 AS TOP3_most_popular_ps,
                           0 AS TOP3_banks
                    FROM Payments
                    JOIN Games
                    ON Games.game_id = Payments.project_id
                    WHERE project_id NOT IN
                    (
                        SELECT project_id
                        FROM Payments
                        WHERE status = '3' AND test_transaction = '0'
                    )



        """
temp = pd.read_sql(sql_code, engine)
temp

,project,successful_transactions,successful_transactions_amount,average_check,sum_amount_nick,TOP3_most_popular_ps,TOP3_banks
0,156. wc,8,553,69.125000,311,"t, x, m","sber, alfa, vtb"
1,164. sims,9,994,110.444444,500,"x, m, t","sber, vtb, alfa"
2,157. wot,0,0,0.000000,0,0,0
3,161. worms,0,0,0.000000,0,0,0
4,165. dota,0,0,0.000000,0,0,0


# Задача 2

В приложенном CSV Файле представлены данные по трафику одного из партнеров. У разработчика две
отдельные игры (Funny squirrel и Funny squirrel in Space). Необходимо выделить наиболее показательные и
релевантные метрики и представить краткий отчет с основными характеристиками трафика игр.

In [181]:
df = pd.read_csv("xsolla")

Немного изучим данные:

In [184]:
df.Month_of_2020.unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [187]:
df['Game name'].unique()

array([' Funny squirrel', ' Funny squirrel in Space'], dtype=object)

In [190]:
df['Payment System'].nunique()

311

In [191]:
df['User country'].nunique()

219

In [219]:
df_game = df.groupby("Game name", as_index = False) \
  .agg({'AFS reject': 'sum', 'Incomplete attempts': 'sum', 'Completed payments': 'sum', 'PS declines': 'sum', 'Refunds': 'sum'})
df_game['All payments'] = df_game['AFS reject']+df_game['Incomplete attempts']+df_game['Completed payments']+df_game['PS declines']+df_game['Refunds']
df_game

,Game name,AFS reject,Incomplete attempts,Completed payments,PS declines,Refunds,All payments
0,Funny squirrel,67731,364379,5233356,683237,29546,6378249
1,Funny squirrel in Space,31946,124554,743509,224696,6718,1131423


In [231]:
df_game['AFS, %'] = round((df_game['AFS reject']/df_game['All payments'])*100, 2)
df_game['Incomplete, %'] = round((df_game['Incomplete attempts']/df_game['All payments'])*100, 2)
df_game['Completed, %'] = round((df_game['Completed payments']/df_game['All payments'])*100, 2)
df_game['PS, %'] = round((df_game['PS declines']/df_game['All payments'])*100, 2)
df_game['Refunds, %'] = round((df_game['Refunds']/df_game['All payments'])*100, 2)
df_game

,Game name,AFS reject,Incomplete attempts,Completed payments,PS declines,Refunds,All payments,"AFS, %","Incomplete, %","Completed, %","PS, %","Refunds, %"
0,Funny squirrel,67731,364379,5233356,683237,29546,6378249,1.06,5.71,82.05,10.71,0.46
1,Funny squirrel in Space,31946,124554,743509,224696,6718,1131423,2.82,11.01,65.71,19.86,0.59


Выводим для каждой игры процент от общего кол-во операций в рамках каждого из показателей:

In [230]:
df_game[['Game name', 'AFS, %', 'Incomplete, %', 'Completed, %', 'PS, %', 'Refunds, %']]

,Game name,"AFS, %","Incomplete, %","Completed, %","PS, %","Refunds, %"
0,Funny squirrel,1.06,5.71,82.05,10.71,0.46
1,Funny squirrel in Space,2.82,11.01,65.71,19.86,0.59


По кол-ву операций игра Funny squirrel сильно популярнее игры Funny squirrel in Space (6.3 млн против 1.1 млн), при этом из общей картины видно, что для Funny squirrel in Space такие "негативные" показатели как доля фрода (почти в 3 раза), доля отказов со стороны платёжной системы (в два раза), доля незавершённых транзакций (в два раза) и доля возвратов ощутимо выше, чем в Funny squirrel. И наоборот, "положительный" показатель - доля успешных платежей (почти на 1/5) выше в игре Funny squirrel.

In [265]:
df_ps = df.groupby("Payment System", as_index = False) \
  .agg({'AFS reject': 'sum', 'Incomplete attempts': 'sum', 'Completed payments': 'sum', 'PS declines': 'sum', 'Refunds': 'sum'})
df_ps['All_payments'] = df_ps['AFS reject']+df_ps['Incomplete attempts']+df_ps['Completed payments']+df_ps['PS declines']+df_ps['Refunds']
df_ps = df_ps.sort_values(by=['All_payments'], ascending=False)

In [266]:
df_ps['AFS, %'] = round((df_ps['AFS reject']/df_ps['All_payments'])*100, 2)
df_ps['Incomplete, %'] = round((df_ps['Incomplete attempts']/df_ps['All_payments'])*100, 2)
df_ps['Completed, %'] = round((df_ps['Completed payments']/df_ps['All_payments'])*100, 2)
df_ps['PS, %'] = round((df_ps['PS declines']/df_ps['All_payments'])*100, 2)
df_ps['Refunds, %'] = round((df_ps['Refunds']/df_ps['All_payments'])*100, 2)

In [267]:
df_ps[['Payment System', 'All_payments', 'AFS, %', 'Incomplete, %', 'Completed, %', 'PS, %', 'Refunds, %']].head(10)

,Payment System,All_payments,"AFS, %","Incomplete, %","Completed, %","PS, %","Refunds, %"
287,Банковские карты,3299068,1.97,1.84,87.34,8.26,0.60
183,PayPal,2292338,0.58,4.15,87.49,7.40,0.39
12,Amazon Pay,220614,4.37,22.23,43.22,29.77,0.41
283,paysafecard,215460,0.93,10.06,75.35,13.57,0.09
225,Sofort EU banking,138310,0.07,9.52,77.80,12.42,0.19
24,BOKU Mobile Commerce,118809,0.12,0.97,77.30,21.52,0.08
152,MyCard Taiwan e-wallet,85006,0.36,22.80,49.76,27.08,0.00
234,TOSS,55066,0.19,0.02,87.98,11.23,0.59
280,iDEAL Netherlands Banking,51442,0.04,5.13,87.05,7.41,0.37
123,KakaoPay,50751,0.02,23.31,48.44,27.83,0.39


Для TOP-10 платёжных систем (по кол-ву операций): бОльшая доля транзакций приходится на 2 платёжные системы банковские карты и PayPal (5.5 млн из всего 7.4 млн), пи этом системы показывают одинаковый результат по успешным платежам. Для 3-х платёжных систем из TOP-10: Amazon Pay, MyCard Taiwan e-wallet и KakaoPay характерно: 1) почти четверть операций не завершается после введения реквизитов из-за того,что пользователь не нажал на кнопку "оплатить"; 2) меньше половины всех операций проходит успешо.


In [268]:
df_month = df.groupby("Month_of_2020", as_index = False) \
  .agg({'AFS reject': 'sum', 'Incomplete attempts': 'sum', 'Completed payments': 'sum', 'PS declines': 'sum', 'Refunds': 'sum'})
df_month['All_payments'] = df_month['AFS reject']+df_month['Incomplete attempts']+df_month['Completed payments']+df_month['PS declines']+df_month['Refunds']
#df_month = df_month.sort_values(by=['All_payments'], ascending=False)

In [269]:
df_month['AFS, %'] = round((df_month['AFS reject']/df_month['All_payments'])*100, 2)
df_month['Incomplete, %'] = round((df_month['Incomplete attempts']/df_month['All_payments'])*100, 2)
df_month['Completed, %'] = round((df_month['Completed payments']/df_month['All_payments'])*100, 2)
df_month['PS, %'] = round((df_month['PS declines']/df_month['All_payments'])*100, 2)
df_month['Refunds, %'] = round((df_month['Refunds']/df_month['All_payments'])*100, 2)

In [270]:
df_month

,Month_of_2020,AFS reject,Incomplete attempts,Completed payments,PS declines,Refunds,All_payments,"AFS, %","Incomplete, %","Completed, %","PS, %","Refunds, %"
0,1,19118,83432,1216316,157492,9782,1486140,1.29,5.61,81.84,10.60,0.66
1,2,19025,80684,1169985,152222,6296,1428212,1.33,5.65,81.92,10.66,0.44
2,3,21375,95212,1167575,175936,6440,1466538,1.46,6.49,79.61,12.00,0.44
3,4,22109,124914,1314985,227758,8633,1698399,1.30,7.35,77.42,13.41,0.51
4,5,18050,104691,1108004,194525,5113,1430383,1.26,7.32,77.46,13.60,0.36


В разрезе месяца трафик распределяется равномерно по всем показателям.